## Modelo 5 - Ensemble de modelos

Aqui, será buscada uma combinação dos modelos testados no notebook anterior, a fim de encontrar um modelo satisfatório, não só em termos de métricas mas também para colocar em produção.

(arquivo ./datasets/raw_data_with_all_labels.xlsx)

In [1]:
#importando pacotes
import pandas as pd
import numpy as np
import re
import time

import bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 100)

#https://strftime.org
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_excel('./datasets/raw_data_with_all_labels.xlsx', index_col=0).dropna(subset=["y"])

In [3]:
df.head()

,watch-title,y,watch-view-count,view,watch-time-text,watch7-headline,watch8-sentiment-actions,og:image,og:image:width,og:description,og:video:width,og:video:height,og:video:tag,content_watch-info-tag-list,channel_link_0
0,Finanças é Coisa de Criança!,0.0,635 visualizações,NaN,Publicado em 30 de jul. de 2019,#intusforma #institucional #educacao\n\n\n\n ...,635 visualizações\n\n\n\n\nGostou deste vídeo?...,https://i.ytimg.com/vi/-0HZcKmHXn4/maxresdefau...,1280,Um dos nossos primeiros programas que deram vi...,1280.0,720.0,financasecoisadecrianca,Educação,/channel/UCjE76zhvoDw5hnLtmzFHR-Q
1,Câncer 2020 - Profissional e Finanças 1° semestre,0.0,37.305 visualizações,NaN,Publicado em 3 de dez. de 2019,#tarô #previsão2020 #profissional2020\n\n\n\n ...,37.305 visualizações\n\n\n\n\n\n\n\n2.550\n\nG...,https://i.ytimg.com/vi/-1kS-ZrfcYQ/hqdefault.jpg,480,Previsão primeiro semestre de 2020 profissiona...,1280.0,720.0,tarot câncer 2020 profissional,Pessoas e blogs,/channel/UCL6feH5A7tjCoNTKAkJ9ITQ
2,Seja Rica: Conquiste sua Independência Financeira,0.0,98 visualizações,NaN,Publicado em 30 de mar. de 2017,Seja Rica: Conquiste sua Independência Financeira,98 visualizações\n\n\n\n\n\n\n\n29\n\nGostou d...,https://i.ytimg.com/vi/-40s-kunvQM/hqdefault.jpg,480,Seja Rica é o novo quadro aqui do canal. Nesse...,640.0,360.0,Rico,Pessoas e blogs,/channel/UChgFe4WPc8Wx4tJVAdd9-rA
3,Independência FINANCEIRA,0.0,41 visualizações,NaN,Publicado em 19 de mar. de 2019,Independência FINANCEIRA,41 visualizações\n\n\n\n\n\n\n\n17\n\nGostou d...,https://i.ytimg.com/vi/-6dz-10LkHc/maxresdefau...,1280,"Como conseguir sua independência financeira, m...",1280.0,720.0,pouco,Pessoas e blogs,/channel/UCUH-FVOIrY6IW2hs66A4IFA
5,6 ERROS para NÃO ter a INDEPENDÊNCIA FINANCEIR...,0.0,389 visualizações,NaN,Publicado em 21 de jan. de 2020,6 ERROS para NÃO ter a INDEPENDÊNCIA FINANCEIR...,389 visualizações\n\n\n\n\n\n\n\n86\n\nGostou ...,https://i.ytimg.com/vi/-CcyxsGoquk/maxresdefau...,1280,"Ano novo, vida financeira nova! A partir de ag...",1280.0,720.0,não ter independência financeira,Educação,/channel/UCu8nmn2Na1wcPUxy7dDqG2A


In [4]:
df.duplicated().mean()

0.0

In [5]:
df.duplicated(['watch-title']).mean()

0.0017667844522968198

In [6]:
df = df[~df.duplicated(['watch-title'], keep=False)]

In [7]:
df.duplicated(['watch-title']).mean()

0.0

In [8]:
df.shape

(564, 15)

In [9]:
#criando dataframe com os índices do df_labeled
df_limpo = pd.DataFrame(index = df.index)

In [10]:
df_limpo['title'] = df['watch-title']

## Limpando os dados

In [11]:
#formato da data de publicação no dataframe com labels:
df['watch-time-text'].head()

0    Publicado em 30 de jul. de 2019
1     Publicado em 3 de dez. de 2019
2    Publicado em 30 de mar. de 2017
3    Publicado em 19 de mar. de 2019
5    Publicado em 21 de jan. de 2020
Name: watch-time-text, dtype: object

In [12]:
map_mes = {'jan.':'Jan',
             'fev.':'Feb',
              'mar.':'Mar',
              'abr.':'Apr',
              'mai.':'May',
              'jun.':'Jun',
              'jul.':'Jul',
              'ago.':'Aug',
              'set.':'Sep',
              'out.':'Oct',
              'nov.':'Nov',
              'dez.':'Dec'}

In [13]:
def limpa_data (row):
    
    data_limpa = re.search(r"(\d+) de ([a-z]+)\. de (\d+)", row.loc['watch-time-text']).group()
    data_limpa = data_limpa.split('de ')
    data_limpa = [x.strip() for x in data_limpa]
    data_limpa[0] = ['0' + str(data_limpa[0]) if len(data_limpa[0]) == 1 else data_limpa[0]][0]
    data_limpa[1] = [v for k, v in map_mes.items() if k == data_limpa[1]][0]
    data = '-'.join(data_limpa)

    return data

In [14]:
df['data'] = df.apply(limpa_data, axis=1)

In [15]:
df_limpo['data'] = pd.to_datetime(df['data']).copy()

In [16]:
#coluna com informação de views
df['watch-view-count'].head()

0       635 visualizações
1    37.305 visualizações
2        98 visualizações
3        41 visualizações
5       389 visualizações
Name: watch-view-count, dtype: object

In [17]:
#adicionando coluna com view count formatado
df_limpo['views'] = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".","").fillna(0).astype(int)

## Criando Features

In [18]:
df_limpo.head()

,title,data,views
0,Finanças é Coisa de Criança!,2019-07-30,635
1,Câncer 2020 - Profissional e Finanças 1° semestre,2019-12-03,37305
2,Seja Rica: Conquiste sua Independência Financeira,2017-03-30,98
3,Independência FINANCEIRA,2019-03-19,41
5,6 ERROS para NÃO ter a INDEPENDÊNCIA FINANCEIR...,2020-01-21,389


In [19]:
#criando dataframe para as features:
features = pd.DataFrame(index=df_limpo.index)
#criando série com as labels
y = df['y'].copy()

In [20]:
#criando coluna com dias que se passaram desde a publicação até a data referência
features['dias_publicado'] = (pd.to_datetime("2020-05-08") - df_limpo['data']) / np.timedelta64(1, 'D') #denominador cria um objeto timedelta do numpy em diferença de 1 dia

#adicionando a coluna de views ao dataframe features
features['views'] = df_limpo['views']

#calculando views por dia
features['views_por_dia'] = features['views']/features['dias_publicado']

# retirando coluna de dias após publicação (para explicação, ver notebook para modelo1)
features = features.drop('dias_publicado', axis = 1)

In [21]:
features.head()

,views,views_por_dia
0,635,2.243816
1,37305,237.611465
2,98,0.086344
3,41,0.098558
5,389,3.601852


In [22]:
#criando sets de treino e validação, separando os dados em 50% (quantil 0.5)
mask_train = df_limpo.data < df_limpo.data.quantile(0.5)
mask_val = df_limpo.data >= df_limpo.data.quantile(0.5)

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((282, 2), (282, 2), (282,), (282,))

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title'] #coletando títulos do set de treino
title_val = df_limpo[mask_val]['title'] #coletando títulos do set de validação

title_vec = TfidfVectorizer(min_df = 2, ngram_range=(1,3)) #vetorizador. lembrar que modelos não trabalham com strings
#ocorrência mínima do termo em 2 vídeos

title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val) #usamos somente transform para transformar somente baseado nas palavras no dataset passado para ele.

In [24]:
title_bow_train.shape  

(282, 497)

Mais sobre Tf iDF [aqui](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).
O resultado acima indica que foram criadas 497 colunas no title_bow_train. Cada coluna é um vetor para cada palavra do título. O formato será uma matriz esparsa (só armazena valores maiores que zero), do scipy.

In [90]:
title_bow_train

<282x497 sparse matrix of type '<class 'numpy.float64'>'
	with 2584 stored elements in Compressed Sparse Row format>

A matriz criada para title_bow_train armazenou 1411 valores na mmemória. Se fosse armazenar todos os valores, teríamos:

Para juntar as features de texto com as numéricas, deve-se usar a função hstack do scipy.sparse (hstack e vstack permitem lidar com matrizes esparsas com mais eficiência.
Vamos juntas as features para poder treinar o novo modelo. Mais sobre scipy sparse [aqui](https://docs.scipy.org/doc/scipy/reference/sparse.html).

In [28]:
from scipy.sparse import hstack, vstack

In [29]:
Xtrain_wtitle = hstack([Xtrain, title_bow_train]) #aqui, juntamos as features de treino em Xtrain com as geradas para os títulos das features de treino
Xval_wtitle = hstack([Xval, title_bow_val]) #ídem para validação

In [30]:
# o total de colunas será a soma das colunas das features numéricas com as de texto (2 + 211 = 213)
Xtrain_wtitle.shape, Xval_wtitle.shape

((282, 499), (282, 499))

## Random Forest

In [34]:
#importando alguns modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

In [51]:
%%time
#treinando modelo
# 1000 árvores

mdl_rf = RandomForestClassifier(n_estimators = 1000, min_samples_leaf=1, random_state = 0, class_weight="balanced", n_jobs=8)
mdl_rf.fit(Xtrain_wtitle, ytrain)

Wall time: 1.27 s


RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=8,
                       random_state=0)

In [52]:
p_rf = mdl_rf.predict_proba(Xval_wtitle)[:, 1]

In [53]:
average_precision_score(yval, p_rf), roc_auc_score(yval, p_rf)

(0.7070328313955678, 0.8872246696035242)

## LightGBM
utilizando os parâmetros tunados no notebook 4.

In [37]:
from lightgbm import LGBMClassifier

In [67]:
#aqui, mudamos o ngram_range para (1,3) para ter o mesmo vetorizados da RF
params = [0.00781968225875022, 3, 4, 0.7078936710077383, 0.31818755505678337, 275, 4, 3]

lr=params[0]
max_depth=params[1]
min_child_samples=params[2]
subsample=params[3] #pega uma amostra dos dados a cada rodada de treino para diminuir overfitting
colsample_bytree=params[4]
n_estimators=params[5]

min_df=params[6]
ngram_range=(1, params[7])

title_vec = TfidfVectorizer(min_df = min_df, ngram_range=ngram_range) 
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val) 

Xtrain_wtitle = hstack([Xtrain, title_bow_train]) 
Xval_wtitle = hstack([Xval, title_bow_val])

mdl_lgbm =LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth = max_depth, min_child_samples = min_child_samples,
                   subsample = subsample, colsample_bytree = colsample_bytree, bagging_freq = 1, n_estimators = n_estimators,
                   random_state=0, class_weight = 'balanced', n_jobs = 8)

mdl_lgbm.fit(Xtrain_wtitle, ytrain)

p_lgbm = mdl_lgbm.predict_proba(Xval_wtitle)[:, 1]
    


d:\solucao completa de ds - mario filho\projeto\.env\lib\site-packages\lightgbm\basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [68]:
average_precision_score(yval, p2), roc_auc_score(yval, p2)

(0.7274743798668423, 0.893311974369243)

## Logistic Regression

In [43]:
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix #converte matrix densa em sparse

In [41]:
from sklearn.pipeline import make_pipeline

In [44]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

In [45]:
lr_pipeline = make_pipeline(MaxAbsScaler(),LogisticRegression(C=0.5, penalty='l2', n_jobs=6, random_state=0))
lr_pipeline.fit(Xtrain_wtitle2, ytrain)

Pipeline(steps=[('maxabsscaler', MaxAbsScaler()),
                ('logisticregression',
                 LogisticRegression(C=0.5, n_jobs=6, random_state=0))])

In [46]:
p_lr = lr_pipeline.predict_proba(Xval_wtitle2)[:,1]

In [47]:
average_precision_score(yval, p_lr), roc_auc_score(yval, p_lr)

(0.5079593263614605, 0.8436523828594313)

## Ensemble

#### anotando todos os scores de cada modelo
(average_precision, roc_auc)  

(0.7094600167650358, 0.8830196235482579) - RF  
(0.7197493613596355, 0.8909891870244293) - LGBM ngram_range (1,4)
(0.7274743798668423, 0.893311974369243) - LGBM ngram_range (1,3)
(0.5079593263614605, 0.8436523828594313) - LR

In [69]:
# média simples
p = (p_lr + p_rf + p_lgbm)/3
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.6980763543070417, 0.8905086103323989)

In [70]:
# verificando diferenças entre os modelos (calculando correlação de pearson)
pd.DataFrame({"LR":p_lr, "RF":p_rf, "LGBM":p_lgbm}).corr()

,LR,RF,LGBM
LR,1.000000,0.830500,0.827342
RF,0.830500,1.000000,0.964848
LGBM,0.827342,0.964848,1.000000


Acima, podemos ver que a correlação entre LGBM e RF é bem alta (acima de 96%). Isso pode indicar que fazer um ensemble dos modelos não geraria ganho que justifique, diferentemente da solução do Mário Filho dentro do curso.

In [71]:
#testando LGBM e RF
p2 = 0.5*p_rf + 0.5*p_lgbm
average_precision_score(yval, p), roc_auc_score(yval, p2)

(0.6980763543070417, 0.8926712054465359)

In [76]:
#testando LGBM e RF
p3 = 0.3*p_rf + 0.7*p_lgbm
average_precision_score(yval, p), roc_auc_score(yval, p3) #há um pouco de overfitting por estarmos fazendo em dados valid (o modelo já viu)

(0.6980763543070417, 0.8929114937925511)

## Salvando modelos

Os testes acima mostraram que nenhuma combinação testada foi capaz de melhorar os scores do LGBM sozinho, motivo pelo qual iremos colocar somente o LGBM em produção.

In [77]:
import joblib as jb

In [79]:
jb.dump(mdl_lgbm, "./modelos/lgbm_20200710.pkl.z")
jb.dump(title_vec, "./modelos/title_vectorizer_20200710.pkl.z")

['./modelos/title_vectorizer_20200710.pkl.z']